# PTB-XL Deep Learning Classification

**Goal:** Beat the classical ML baseline (Macro F1 ≈ 0.69) using deep learning.

---

## Target Superclasses

| Code | Description |
|------|-------------|
| **NORM** | Normal ECG |
| **MI** | Myocardial Infarction |
| **STTC** | ST/T Changes |
| **CD** | Conduction Disturbance |
| **HYP** | Hypertrophy |

---

## Models

| Model | Architecture | Description |
|-------|--------------|-------------|
| **Model A** | 1D CNN | Residual conv blocks + GAP |
| **Model B** | CNN + BiLSTM | CNN features → BiLSTM temporal |
| **Model C** | CNN + Attention | Channel/temporal attention |

---

## Key Differences from Baseline

- **100 Hz signals** — optimized for local training (5x faster than 500 Hz)
- **Raw waveforms** — no hand-crafted features
- **Deep learning** — learns hierarchical representations
- **Official splits** — strat_fold 1-8/9/10 for train/val/test
- **MPS Support** — Apple Silicon GPU acceleration when available

## 🚀 Local Setup

Verify dependencies and check for GPU/MPS acceleration.


In [1]:
# ============================================================
# LOCAL SETUP
# ============================================================

import torch

# Check for hardware acceleration
if torch.cuda.is_available():
    print(f'✅ CUDA GPU available: {torch.cuda.get_device_name(0)}')
    print(f'   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
elif torch.backends.mps.is_available():
    print('✅ Apple MPS (Metal) acceleration available!')
    print('   Using Apple Silicon GPU for faster training.')
else:
    print('⚠️ No GPU detected - using CPU (training will be slower)')

# Verify wfdb is installed
try:
    import wfdb
    print('✅ wfdb package available')
except ImportError:
    print('❌ wfdb not installed. Run: pip install wfdb')

print('\n✅ Local setup complete!')


⚠️ No GPU detected - using CPU (training will be slower)
✅ wfdb package available

✅ Local setup complete!


## 1. Imports & Configuration

In [2]:
# ============================================================
# IMPORTS
# ============================================================

import os
import ast
import warnings
from pathlib import Path
from time import time

import numpy as np
import pandas as pd
import wfdb

from scipy import signal as scipy_signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

from tqdm.auto import tqdm

import matplotlib.pyplot as plt

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print('All imports successful!')

/Volumes/Crucial X6/medical_ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
All imports successful!


In [3]:
# ============================================================
# CONFIGURATION (OPTIMIZED FOR LOCAL)
# ============================================================

# Paths (Local)
DATA_PATH = Path('/Volumes/Crucial X6/medical_ai/ptb-xl/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1')
OUTPUT_PATH = Path('/Volumes/Crucial X6/medical_ai/ptb-xl/outputs_dl')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Verify path exists
if DATA_PATH.exists():
    print(f'✅ Data path found: {DATA_PATH}')
else:
    print(f'❌ Data path NOT found: {DATA_PATH}')
    print('   Please check the path!')

# Target superclasses
SUPERCLASSES = ['NORM', 'MI', 'STTC', 'CD', 'HYP']
N_CLASSES = len(SUPERCLASSES)

# ECG parameters - OPTIMIZED FOR LOCAL (5x faster, 5x less memory)
SAMPLING_RATE = 100  # Hz (using records100/ instead of records500/)
DURATION = 10  # seconds
SEQ_LEN = SAMPLING_RATE * DURATION  # 1000 samples (was 5000)
N_LEADS = 12

# Training parameters - OPTIMIZED FOR LOCAL
BATCH_SIZE = 32  # Reduced for CPU/MPS memory (was 64)
EPOCHS = 50
LEARNING_RATE = 1e-3
PATIENCE = 10

# Baseline to beat
BASELINE_MACRO_F1 = 0.69

print(f'Data path: {DATA_PATH}')
print(f'Sampling rate: {SAMPLING_RATE} Hz (using records100/)')
print(f'Sequence length: {SEQ_LEN} samples')
print(f'Batch size: {BATCH_SIZE}')
print(f'Baseline Macro F1 to beat: {BASELINE_MACRO_F1}')
print(f'\n⚡ Local optimization: Using 100 Hz signals for 5x faster training')

✅ Data path found: /Volumes/Crucial X6/medical_ai/ptb-xl/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1
Data path: /Volumes/Crucial X6/medical_ai/ptb-xl/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1
Sampling rate: 100 Hz (using records100/)
Sequence length: 1000 samples
Batch size: 32
Baseline Macro F1 to beat: 0.69

⚡ Local optimization: Using 100 Hz signals for 5x faster training


## 2. Target Construction

Using official PTB-XL splits:
- **Train:** strat_fold 1-8
- **Validation:** strat_fold 9
- **Test:** strat_fold 10

In [4]:
# ============================================================
# LOAD METADATA & CREATE TARGETS
# ============================================================

# Load database
df = pd.read_csv(DATA_PATH / 'ptbxl_database.csv')
print(f'Loaded {len(df):,} ECG records')

# Parse scp_codes
def parse_scp_codes(scp_str):
    try:
        return ast.literal_eval(scp_str)
    except:
        return {}

df['scp_codes_dict'] = df['scp_codes'].apply(parse_scp_codes)

# Load SCP statements
scp_df = pd.read_csv(DATA_PATH / 'scp_statements.csv', index_col=0)
scp_diagnostic = scp_df[scp_df['diagnostic'] == 1.0]
scp_to_superclass = scp_diagnostic['diagnostic_class'].to_dict()

print(f'Diagnostic SCP codes: {len(scp_to_superclass)}')

Loaded 21,837 ECG records
Diagnostic SCP codes: 44


In [5]:
# ============================================================
# CREATE MULTI-LABEL TARGETS
# ============================================================

def get_superclasses(scp_codes_dict):
    active = set()
    for scp_code, likelihood in scp_codes_dict.items():
        if likelihood > 0 and scp_code in scp_to_superclass:
            superclass = scp_to_superclass[scp_code]
            if superclass in SUPERCLASSES:
                active.add(superclass)
    return list(active)

df['superclasses'] = df['scp_codes_dict'].apply(get_superclasses)

# Filter to ECGs with at least one diagnostic label
df_filtered = df[df['superclasses'].apply(len) > 0].copy()
print(f'ECGs with diagnostic labels: {len(df_filtered):,}')

# Create binary label matrix
mlb = MultiLabelBinarizer(classes=SUPERCLASSES)
y_all = mlb.fit_transform(df_filtered['superclasses'])

print(f'Label matrix shape: {y_all.shape}')

ECGs with diagnostic labels: 21,417
Label matrix shape: (21417, 5)


In [6]:
# ============================================================
# OFFICIAL PTB-XL SPLITS
# ============================================================

# Split by strat_fold
train_mask = df_filtered['strat_fold'].isin([1, 2, 3, 4, 5, 6, 7, 8])
val_mask = df_filtered['strat_fold'] == 9
test_mask = df_filtered['strat_fold'] == 10

df_train = df_filtered[train_mask].reset_index(drop=True)
df_val = df_filtered[val_mask].reset_index(drop=True)
df_test = df_filtered[test_mask].reset_index(drop=True)

y_train = y_all[train_mask.values]
y_val = y_all[val_mask.values]
y_test = y_all[test_mask.values]

print('=' * 60)
print('OFFICIAL PTB-XL SPLITS')
print('=' * 60)
print(f'Train (folds 1-8): {len(df_train):,} samples')
print(f'Val   (fold 9):    {len(df_val):,} samples')
print(f'Test  (fold 10):   {len(df_test):,} samples')

# Class distribution
print('\nClass distribution (Train):')
for i, cls in enumerate(SUPERCLASSES):
    count = y_train[:, i].sum()
    pct = 100 * count / len(y_train)
    print(f'  {cls}: {count:,} ({pct:.1f}%)')

OFFICIAL PTB-XL SPLITS
Train (folds 1-8): 17,100 samples
Val   (fold 9):    2,155 samples
Test  (fold 10):   2,162 samples

Class distribution (Train):
  NORM: 7,607 (44.5%)
  MI: 4,389 (25.7%)
  STTC: 4,094 (23.9%)
  CD: 3,912 (22.9%)
  HYP: 2,121 (12.4%)


In [7]:
# ============================================================
# COMPUTE CLASS WEIGHTS
# ============================================================

# For handling class imbalance
class_counts = y_train.sum(axis=0)
total = len(y_train)
class_weights = total / (N_CLASSES * class_counts)
class_weights = torch.FloatTensor(class_weights).to(DEVICE)

print('Class weights for imbalance handling:')
for cls, w in zip(SUPERCLASSES, class_weights.cpu().numpy()):
    print(f'  {cls}: {w:.3f}')

Class weights for imbalance handling:
  NORM: 0.450
  MI: 0.779
  STTC: 0.835
  CD: 0.874
  HYP: 1.612


## 3. Signal Loading & Dataset

**⚡ Optimized for Colab:** Pre-load ALL signals into memory once to avoid slow Google Drive I/O during training.

- Load 500 Hz ECG signals (all at once)
- Per-lead z-score normalization
- Optional bandpass filter
- In-memory dataset for fast batching

In [8]:
# ============================================================
# PRE-LOAD ALL ECG SIGNALS INTO MEMORY
# ============================================================
# This is CRITICAL for Colab performance - loading from Google Drive
# during training is extremely slow. Pre-loading takes a few minutes
# but makes each epoch ~30x faster.

def bandpass_filter(ecg, sampling_rate=500, lowcut=0.5, highcut=40):
    """Apply bandpass filter to ECG signal."""
    nyq = 0.5 * sampling_rate
    low = lowcut / nyq
    high = highcut / nyq
    b, a = scipy_signal.butter(3, [low, high], btype='band')
    filtered = scipy_signal.filtfilt(b, a, ecg, axis=0)
    return filtered

def load_all_signals(df, data_path, sampling_rate=500, seq_len=5000, 
                     normalize=True, apply_bandpass=True):
    """
    Pre-load ALL ECG signals into a single numpy array.
    This avoids slow Google Drive I/O during training.
    """
    signals = []
    failed = 0
    
    # filename_hr/filename_lr already contains the full relative path (e.g., "records100/00000/00001_lr")
    filename_col = 'filename_hr' if sampling_rate == 500 else 'filename_lr'
    
    for idx in tqdm(range(len(df)), desc=f"Loading {len(df)} ECGs"):
        row = df.iloc[idx]
        filepath = str(data_path / row[filename_col])
        
        try:
            record = wfdb.rdrecord(filepath)
            ecg = record.p_signal  # (time, 12)
            
            # Ensure correct length
            if len(ecg) < seq_len:
                ecg = np.pad(ecg, ((0, seq_len - len(ecg)), (0, 0)))
            elif len(ecg) > seq_len:
                ecg = ecg[:seq_len]
            
            # Bandpass filter
            if apply_bandpass:
                try:
                    ecg = bandpass_filter(ecg, sampling_rate)
                except:
                    pass
            
            # Per-lead normalization
            if normalize:
                mean = ecg.mean(axis=0, keepdims=True)
                std = ecg.std(axis=0, keepdims=True) + 1e-8
                ecg = (ecg - mean) / std
            
            # Convert to (channels, time) for Conv1D
            ecg = ecg.T  # (12, seq_len)
            signals.append(ecg)
            
        except Exception as e:
            # Fallback to zeros if loading fails
            signals.append(np.zeros((12, seq_len)))
            failed += 1
    
    signals_array = np.array(signals, dtype=np.float32)
    print(f"✅ Loaded {len(signals)} ECGs ({failed} failed)")
    print(f"   Memory: {signals_array.nbytes / 1e9:.2f} GB")
    
    return signals_array

print('Pre-loading function defined.')

Pre-loading function defined.


In [9]:
# ============================================================
# PRE-LOAD ALL DATA INTO MEMORY
# ============================================================
# This takes ~5-10 minutes but makes training MUCH faster!

print("=" * 60)
print("PRE-LOADING ALL ECG SIGNALS INTO MEMORY")
print("This will take a few minutes, but training will be ~30x faster!")
print("=" * 60)

X_train = load_all_signals(df_train, DATA_PATH, SAMPLING_RATE, SEQ_LEN)
X_val = load_all_signals(df_val, DATA_PATH, SAMPLING_RATE, SEQ_LEN)
X_test = load_all_signals(df_test, DATA_PATH, SAMPLING_RATE, SEQ_LEN)

print(f"\n✅ All data loaded!")
print(f"   X_train: {X_train.shape} ({X_train.nbytes/1e9:.2f} GB)")
print(f"   X_val:   {X_val.shape} ({X_val.nbytes/1e9:.2f} GB)")
print(f"   X_test:  {X_test.shape} ({X_test.nbytes/1e9:.2f} GB)")
print(f"   Total:   {(X_train.nbytes + X_val.nbytes + X_test.nbytes)/1e9:.2f} GB")

PRE-LOADING ALL ECG SIGNALS INTO MEMORY
This will take a few minutes, but training will be ~30x faster!


Loading 17100 ECGs:   0%|          | 0/17100 [00:00<?, ?it/s]

Loading 17100 ECGs: 100%|██████████| 17100/17100 [02:45<00:00, 103.12it/s]


✅ Loaded 17100 ECGs (0 failed)
   Memory: 0.82 GB


Loading 2155 ECGs: 100%|██████████| 2155/2155 [00:21<00:00, 98.00it/s] 


✅ Loaded 2155 ECGs (0 failed)
   Memory: 0.10 GB


Loading 2162 ECGs: 100%|██████████| 2162/2162 [00:20<00:00, 107.16it/s]


✅ Loaded 2162 ECGs (0 failed)
   Memory: 0.10 GB

✅ All data loaded!
   X_train: (17100, 12, 1000) (0.82 GB)
   X_val:   (2155, 12, 1000) (0.10 GB)
   X_test:  (2162, 12, 1000) (0.10 GB)
   Total:   1.03 GB


In [10]:
# ============================================================
# FAST IN-MEMORY DATASET & DATALOADERS
# ============================================================

class FastECGDataset(Dataset):
    """Simple in-memory dataset for pre-loaded signals."""
    def __init__(self, signals, labels):
        self.signals = torch.FloatTensor(signals)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.signals)
    
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# Create datasets from pre-loaded data
train_dataset = FastECGDataset(X_train, y_train)
val_dataset = FastECGDataset(X_val, y_val)
test_dataset = FastECGDataset(X_test, y_test)

# Optimized DataLoaders
# - num_workers=0 for parallel loading
# - pin_memory=True for faster GPU transfer
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=0)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')
print(f'Test batches: {len(test_loader)}')

# Test loading speed
import time
t0 = time.time()
X_sample, y_sample = next(iter(train_loader))
print(f'\n⚡ Sample batch loaded in {(time.time()-t0)*1000:.1f}ms')
print(f'   X={X_sample.shape}, y={y_sample.shape}')


Train batches: 535
Val batches: 68
Test batches: 68

⚡ Sample batch loaded in 187.2ms
   X=torch.Size([32, 12, 1000]), y=torch.Size([32, 5])


## 4. Model Architectures

### Model A: 1D CNN (Residual Blocks)
- Temporal convolutions with skip connections
- Global Average Pooling
- Fully connected classifier

### Model B: CNN + BiLSTM
- CNN for local feature extraction
- BiLSTM for temporal dependencies

### Model C: CNN + Attention
- CNN backbone
- Multi-head self-attention

In [12]:
# ============================================================
# MODEL A: 1D CNN WITH RESIDUAL BLOCKS
# ============================================================

class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=7, stride=1):
        super().__init__()
        padding = kernel_size // 2
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride),
                nn.BatchNorm1d(out_channels)
            )
        
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.dropout(out)
        out += self.shortcut(x)
        return F.relu(out)

class CNN1D(nn.Module):
    def __init__(self, n_leads=12, n_classes=5, seq_len=5000):
        super().__init__()
        
        # Initial convolution
        self.conv1 = nn.Conv1d(n_leads, 32, kernel_size=15, padding=7)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(2)
        
        # Residual blocks
        self.res1 = ResidualBlock1D(32, 64, stride=2)
        self.res2 = ResidualBlock1D(64, 128, stride=2)
        self.res3 = ResidualBlock1D(128, 256, stride=2)
        self.res4 = ResidualBlock1D(256, 256, stride=2)
        
        # Global pooling
        self.gap = nn.AdaptiveAvgPool1d(1)
        
        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, n_classes)
        )
    
    def forward(self, x):
        # x: (batch, 12, 5000)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x

# Test model
model_test = CNN1D().to(DEVICE)
with torch.no_grad():
    out = model_test(X_sample.to(DEVICE))
print(f'CNN1D output shape: {out.shape}')
print(f'CNN1D parameters: {sum(p.numel() for p in model_test.parameters()):,}')

CNN1D output shape: torch.Size([32, 5])
CNN1D parameters: 1,974,949


In [13]:
# ============================================================
# MODEL B: CNN + BiLSTM
# ============================================================

class CNN_BiLSTM(nn.Module):
    def __init__(self, n_leads=12, n_classes=5, seq_len=5000):
        super().__init__()
        
        # CNN feature extractor
        self.cnn = nn.Sequential(
            nn.Conv1d(n_leads, 32, kernel_size=15, padding=7),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(4),
            
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(4),
        )
        
        # BiLSTM
        self.lstm = nn.LSTM(128, 64, num_layers=2, batch_first=True, 
                           bidirectional=True, dropout=0.3)
        
        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, n_classes)
        )
    
    def forward(self, x):
        # x: (batch, 12, 5000)
        x = self.cnn(x)  # (batch, 128, seq)
        x = x.permute(0, 2, 1)  # (batch, seq, 128)
        
        x, _ = self.lstm(x)  # (batch, seq, 128)
        x = x[:, -1, :]  # Last timestep
        
        x = self.fc(x)
        return x

# Test model
model_test = CNN_BiLSTM().to(DEVICE)
with torch.no_grad():
    out = model_test(X_sample.to(DEVICE))
print(f'CNN_BiLSTM output shape: {out.shape}')
print(f'CNN_BiLSTM parameters: {sum(p.numel() for p in model_test.parameters()):,}')

CNN_BiLSTM output shape: torch.Size([32, 5])
CNN_BiLSTM parameters: 268,965


In [14]:
# ============================================================
# MODEL C: CNN + ATTENTION
# ============================================================

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # x: (batch, channels, time)
        avg = self.avg_pool(x).squeeze(-1)
        max_ = self.max_pool(x).squeeze(-1)
        attn = self.fc(avg) + self.fc(max_)
        return x * attn.unsqueeze(-1)

class TemporalAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.query = nn.Linear(channels, channels // 4)
        self.key = nn.Linear(channels, channels // 4)
        self.value = nn.Linear(channels, channels)
        self.scale = (channels // 4) ** -0.5
    
    def forward(self, x):
        # x: (batch, time, channels)
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        attn = F.softmax(attn, dim=-1)
        
        out = torch.matmul(attn, v)
        return out + x

class CNN_Attention(nn.Module):
    def __init__(self, n_leads=12, n_classes=5, seq_len=5000):
        super().__init__()
        
        # CNN with channel attention
        self.conv1 = nn.Conv1d(n_leads, 32, kernel_size=15, padding=7)
        self.bn1 = nn.BatchNorm1d(32)
        self.ca1 = ChannelAttention(32)
        self.pool1 = nn.MaxPool1d(4)
        
        self.conv2 = nn.Conv1d(32, 64, kernel_size=7, padding=3)
        self.bn2 = nn.BatchNorm1d(64)
        self.ca2 = ChannelAttention(64)
        self.pool2 = nn.MaxPool1d(4)
        
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.ca3 = ChannelAttention(128)
        self.pool3 = nn.MaxPool1d(4)
        
        # Temporal attention
        self.temporal_attn = TemporalAttention(128)
        
        # Global pooling + classifier
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, n_classes)
        )
    
    def forward(self, x):
        # x: (batch, 12, 5000)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.ca1(x)
        x = self.pool1(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.ca2(x)
        x = self.pool2(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.ca3(x)
        x = self.pool3(x)
        
        # Temporal attention
        x = x.permute(0, 2, 1)  # (batch, time, channels)
        x = self.temporal_attn(x)
        x = x.permute(0, 2, 1)  # (batch, channels, time)
        
        x = self.gap(x).squeeze(-1)
        x = self.fc(x)
        return x

# Test model
model_test = CNN_Attention().to(DEVICE)
with torch.no_grad():
    out = model_test(X_sample.to(DEVICE))
print(f'CNN_Attention output shape: {out.shape}')
print(f'CNN_Attention parameters: {sum(p.numel() for p in model_test.parameters()):,}')

CNN_Attention output shape: torch.Size([32, 5])
CNN_Attention parameters: 106,109


## 5. Training Setup

- Binary Cross-Entropy loss with class weights
- Adam optimizer with learning rate scheduling
- Early stopping on validation Macro F1

In [15]:
# ============================================================
# TRAINING FUNCTIONS (with progress bars)
# ============================================================

def train_epoch(model, loader, optimizer, criterion, epoch, epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    # Progress bar for batches
    pbar = tqdm(loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', 
                leave=False, ncols=100)
    
    for batch_idx, (X, y) in enumerate(pbar):
        X, y = X.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.sigmoid(outputs).cpu().detach().numpy()
        all_preds.append(preds)
        all_labels.append(y.cpu().numpy())
        
        # Update progress bar with current loss
        avg_loss = total_loss / (batch_idx + 1)
        pbar.set_postfix({'loss': f'{avg_loss:.4f}'})
    
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # Threshold at 0.5
    pred_binary = (all_preds > 0.5).astype(int)
    macro_f1 = f1_score(all_labels, pred_binary, average='macro', zero_division=0)
    
    return total_loss / len(loader), macro_f1

def evaluate(model, loader, criterion, desc='Val'):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(loader, desc=f'         [{desc}]', leave=False, ncols=100)
    
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(DEVICE), y.to(DEVICE)
            outputs = model(X)
            loss = criterion(outputs, y)
            
            total_loss += loss.item()
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(y.cpu().numpy())
    
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    pred_binary = (all_preds > 0.5).astype(int)
    macro_f1 = f1_score(all_labels, pred_binary, average='macro', zero_division=0)
    
    return total_loss / len(loader), macro_f1, all_preds, all_labels

print('Training functions defined (with tqdm progress bars).')

Training functions defined (with tqdm progress bars).


In [16]:
# ============================================================
# TRAINING LOOP
# ============================================================

def train_model(model, model_name, train_loader, val_loader, epochs=50, patience=10, lr=1e-3):
    print(f'\n{"="*60}')
    print(f'TRAINING: {model_name}')
    print(f'{"="*60}')
    
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
    
    history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': []}
    best_val_f1 = 0
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(epochs):
        t0 = time.time()
        
        train_loss, train_f1 = train_epoch(model, train_loader, optimizer, criterion, epoch, epochs)
        val_loss, val_f1, _, _ = evaluate(model, val_loader, criterion, desc='Val')
        
        scheduler.step(val_f1)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)
        
        elapsed = time.time() - t0
        
        # Check for improvement
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            marker = ' ★'
        else:
            patience_counter += 1
            marker = ''
        
        print(f'Epoch {epoch+1:2d}/{epochs} | '
              f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | '
              f'Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f} | '
              f'{elapsed:.1f}s{marker}')
        
        # Early stopping
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f'\nBest Val F1: {best_val_f1:.4f}')
    
    return model, history, best_val_f1

print('Training loop defined.')

Training loop defined.


## 6. Train All Models

In [17]:
# ============================================================
# TRAIN MODEL A: 1D CNN
# ============================================================

model_a = CNN1D(n_leads=N_LEADS, n_classes=N_CLASSES, seq_len=SEQ_LEN).to(DEVICE)
model_a, history_a, best_f1_a = train_model(
    model_a, 'Model A: 1D CNN', 
    train_loader, val_loader, 
    epochs=EPOCHS, patience=PATIENCE, lr=LEARNING_RATE, 
)


TRAINING: Model A: 1D CNN


TypeError: 'module' object is not callable

In [ ]:
# ============================================================
# TRAIN MODEL B: CNN + BiLSTM
# ============================================================

model_b = CNN_BiLSTM(n_leads=N_LEADS, n_classes=N_CLASSES, seq_len=SEQ_LEN).to(DEVICE)
model_b, history_b, best_f1_b = train_model(
    model_b, 'Model B: CNN + BiLSTM', 
    train_loader, val_loader, 
    epochs=EPOCHS, patience=PATIENCE, lr=LEARNING_RATE
)

In [ ]:
# ============================================================
# TRAIN MODEL C: CNN + ATTENTION
# ============================================================

model_c = CNN_Attention(n_leads=N_LEADS, n_classes=N_CLASSES, seq_len=SEQ_LEN).to(DEVICE)
model_c, history_c, best_f1_c = train_model(
    model_c, 'Model C: CNN + Attention', 
    train_loader, val_loader, 
    epochs=EPOCHS, patience=PATIENCE, lr=LEARNING_RATE
)

## 7. Test Set Evaluation

In [ ]:
# ============================================================
# EVALUATE ON TEST SET
# ============================================================

def compute_test_metrics(model, model_name, test_loader):
    print(f'\nEvaluating {model_name}...')
    criterion = nn.BCEWithLogitsLoss()
    _, _, preds, labels = evaluate(model, test_loader, criterion, desc='Test')
    
    pred_binary = (preds > 0.5).astype(int)
    
    metrics = {
        'model': model_name,
        'macro_f1': f1_score(labels, pred_binary, average='macro', zero_division=0),
        'micro_f1': f1_score(labels, pred_binary, average='micro', zero_division=0),
    }
    
    # Per-class metrics
    for i, cls in enumerate(SUPERCLASSES):
        metrics[f'precision_{cls}'] = precision_score(labels[:, i], pred_binary[:, i], zero_division=0)
        metrics[f'recall_{cls}'] = recall_score(labels[:, i], pred_binary[:, i], zero_division=0)
        metrics[f'f1_{cls}'] = f1_score(labels[:, i], pred_binary[:, i], zero_division=0)
        try:
            metrics[f'auroc_{cls}'] = roc_auc_score(labels[:, i], preds[:, i])
        except:
            metrics[f'auroc_{cls}'] = np.nan
    
    return metrics, preds, labels

# Evaluate all models
results_a, preds_a, labels_a = compute_test_metrics(model_a, 'CNN1D', test_loader)
results_b, preds_b, labels_b = compute_test_metrics(model_b, 'CNN+BiLSTM', test_loader)
results_c, preds_c, labels_c = compute_test_metrics(model_c, 'CNN+Attention', test_loader)

print('Test evaluation complete!')

In [ ]:
# ============================================================
# RESULTS COMPARISON
# ============================================================

print('\n' + '=' * 80)
print('TEST SET RESULTS')
print('=' * 80)

# Summary table
results_df = pd.DataFrame([
    {'Model': 'Baseline (ML)', 'Macro F1': BASELINE_MACRO_F1, 'Micro F1': '-'},
    {'Model': 'CNN1D', 'Macro F1': f"{results_a['macro_f1']:.4f}", 'Micro F1': f"{results_a['micro_f1']:.4f}"},
    {'Model': 'CNN+BiLSTM', 'Macro F1': f"{results_b['macro_f1']:.4f}", 'Micro F1': f"{results_b['micro_f1']:.4f}"},
    {'Model': 'CNN+Attention', 'Macro F1': f"{results_c['macro_f1']:.4f}", 'Micro F1': f"{results_c['micro_f1']:.4f}"},
])

print('\n📊 MODEL COMPARISON:')
print(results_df.to_string(index=False))

# Find best model
best_result = max([results_a, results_b, results_c], key=lambda x: x['macro_f1'])
print(f'\n🏆 BEST MODEL: {best_result["model"]} (Macro F1 = {best_result["macro_f1"]:.4f})')

# Compare to baseline
improvement = best_result['macro_f1'] - BASELINE_MACRO_F1
if improvement > 0:
    print(f'✅ BEATS BASELINE by +{improvement:.4f}')
else:
    print(f'❌ Below baseline by {improvement:.4f}')

In [ ]:
# ============================================================
# PER-CLASS PERFORMANCE (BEST MODEL)
# ============================================================

print('\n' + '=' * 80)
print(f'PER-CLASS METRICS ({best_result["model"]})')
print('=' * 80)

per_class_df = pd.DataFrame([
    {
        'Class': cls,
        'Precision': f"{best_result[f'precision_{cls}']:.4f}",
        'Recall': f"{best_result[f'recall_{cls}']:.4f}",
        'F1': f"{best_result[f'f1_{cls}']:.4f}",
        'AUROC': f"{best_result[f'auroc_{cls}']:.4f}"
    }
    for cls in SUPERCLASSES
])
print(per_class_df.to_string(index=False))

# Compare all models per class
print('\n' + '=' * 80)
print('PER-CLASS F1 COMPARISON')
print('=' * 80)
print(f'{"Class":<8} {"CNN1D":<10} {"CNN+BiLSTM":<12} {"CNN+Attention":<14}')
print('-' * 50)
for cls in SUPERCLASSES:
    print(f'{cls:<8} {results_a[f"f1_{cls}"]:<10.4f} {results_b[f"f1_{cls}"]:<12.4f} {results_c[f"f1_{cls}"]:<14.4f}')

## 8. Training Curves & Visualization

In [ ]:
# ============================================================
# TRAINING CURVES
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

histories = [history_a, history_b, history_c]
names = ['CNN1D', 'CNN+BiLSTM', 'CNN+Attention']

for i, (hist, name) in enumerate(zip(histories, names)):
    # Loss
    axes[0, i].plot(hist['train_loss'], label='Train', linewidth=2)
    axes[0, i].plot(hist['val_loss'], label='Val', linewidth=2)
    axes[0, i].set_xlabel('Epoch')
    axes[0, i].set_ylabel('Loss')
    axes[0, i].set_title(f'{name} - Loss', fontweight='bold')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)
    
    # F1
    axes[1, i].plot(hist['train_f1'], label='Train', linewidth=2)
    axes[1, i].plot(hist['val_f1'], label='Val', linewidth=2)
    axes[1, i].axhline(y=BASELINE_MACRO_F1, color='r', linestyle='--', label='Baseline')
    axes[1, i].set_xlabel('Epoch')
    axes[1, i].set_ylabel('Macro F1')
    axes[1, i].set_title(f'{name} - Macro F1', fontweight='bold')
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'training_curves.png', dpi=150)
plt.show()

In [ ]:
# ============================================================
# MODEL COMPARISON BAR CHART
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Macro F1 comparison
models = ['Baseline', 'CNN1D', 'CNN+BiLSTM', 'CNN+Attention']
macro_f1s = [BASELINE_MACRO_F1, results_a['macro_f1'], results_b['macro_f1'], results_c['macro_f1']]
colors = ['#95a5a6' if f < BASELINE_MACRO_F1 else '#2ecc71' for f in macro_f1s]
colors[0] = '#3498db'  # Baseline

bars = axes[0].bar(models, macro_f1s, color=colors, edgecolor='black')
axes[0].axhline(y=BASELINE_MACRO_F1, color='red', linestyle='--', linewidth=2, label='Baseline')
axes[0].set_ylabel('Macro F1')
axes[0].set_title('Model Comparison (Macro F1)', fontweight='bold')
axes[0].set_ylim([0.5, 0.85])
for bar, f1 in zip(bars, macro_f1s):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{f1:.3f}', ha='center', fontsize=11, fontweight='bold')

# Per-class F1 for best model
class_f1s = [best_result[f'f1_{cls}'] for cls in SUPERCLASSES]
class_colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12']
bars = axes[1].bar(SUPERCLASSES, class_f1s, color=class_colors, edgecolor='black')
axes[1].set_ylabel('F1 Score')
axes[1].set_title(f'Per-Class F1 ({best_result["model"]})', fontweight='bold')
axes[1].set_ylim([0, 1])
for bar, f1 in zip(bars, class_f1s):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'{f1:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'model_comparison.png', dpi=150)
plt.show()

## 9. Analysis & Discussion

In [ ]:
# ============================================================
# ANALYSIS
# ============================================================

print('=' * 80)
print('ANALYSIS & DISCUSSION')
print('=' * 80)

print(f'''
1. PERFORMANCE SUMMARY
   ====================
   
   Baseline (ML):     Macro F1 = {BASELINE_MACRO_F1:.4f}
   CNN1D:             Macro F1 = {results_a['macro_f1']:.4f}
   CNN+BiLSTM:        Macro F1 = {results_b['macro_f1']:.4f}
   CNN+Attention:     Macro F1 = {results_c['macro_f1']:.4f}
   
   Best Model: {best_result['model']}
   Improvement: {'+' if best_result['macro_f1'] > BASELINE_MACRO_F1 else ''}{(best_result['macro_f1'] - BASELINE_MACRO_F1):.4f}

2. PER-CLASS ANALYSIS
   ====================
   
   NORM: F1 = {best_result['f1_NORM']:.4f}
   - Strong performance (largest class, clear pattern)
   
   MI:   F1 = {best_result['f1_MI']:.4f}
   - Deep learning captures ST-segment/Q-wave morphology well
   
   STTC: F1 = {best_result['f1_STTC']:.4f}
   - Challenging due to overlap with other conditions
   
   CD:   F1 = {best_result['f1_CD']:.4f}
   - Conduction patterns (BBB, AV blocks) are distinct
   
   HYP:  F1 = {best_result['f1_HYP']:.4f}
   - Voltage criteria benefit from raw signal processing

3. 500 Hz vs 100 Hz
   =================
   
   - Higher resolution captures subtle morphological details
   - Better for HF components of QRS complex
   - Increased computational cost (5x more samples)
   - Beneficial for CD and MI detection

4. MODEL COMPLEXITY
   =================
   
   CNN1D:        {sum(p.numel() for p in model_a.parameters()):,} parameters
   CNN+BiLSTM:   {sum(p.numel() for p in model_b.parameters()):,} parameters
   CNN+Attention: {sum(p.numel() for p in model_c.parameters()):,} parameters
   
   Trade-off: More complex models may overfit on PTB-XL size (~18k train samples)

5. RECOMMENDATIONS
   ================
   
   - CNN1D is efficient and performs well
   - Attention helps with temporal dependencies
   - BiLSTM may be overkill for this dataset size
   - Consider ensemble of models for production
   - Focus on improving STTC and HYP classes
''')

## 10. Final Summary

In [ ]:
# ============================================================
# FINAL SUMMARY
# ============================================================

print('=' * 80)
print('🎯 PTB-XL DEEP LEARNING CLASSIFICATION - FINAL SUMMARY')
print('=' * 80)

print(f'''
DATASET:
  Total ECGs: {len(df_filtered):,}
  Train/Val/Test: {len(df_train):,} / {len(df_val):,} / {len(df_test):,}
  Sampling Rate: {SAMPLING_RATE} Hz
  Sequence Length: {SEQ_LEN} samples

MODELS TRAINED:
  1. CNN1D (Residual blocks)
  2. CNN+BiLSTM (Temporal modeling)
  3. CNN+Attention (Channel + Temporal attention)

RESULTS:
''')
print(results_df.to_string(index=False))

print(f'''
BEST MODEL: {best_result['model']}
  Macro F1: {best_result['macro_f1']:.4f}
  vs Baseline: {'+' if best_result['macro_f1'] > BASELINE_MACRO_F1 else ''}{(best_result['macro_f1'] - BASELINE_MACRO_F1):.4f}

KEY INSIGHTS:
  ✓ Deep learning can match/beat classical ML on PTB-XL
  ✓ 500 Hz signals provide better morphological detail
  ✓ Raw waveforms work well without hand-crafted features
  ✓ Attention mechanisms help with interpretability
  
NEXT STEPS:
  1. Try larger models (ResNet-18, EfficientNet-1D)
  2. Multi-task learning with rhythm labels
  3. Data augmentation (time warping, noise injection)
  4. Ensemble methods
  5. External validation on other datasets
''')

In [ ]:
# ============================================================
# SAVE MODELS & RESULTS
# ============================================================

import json

# Save best model
torch.save(model_a.state_dict(), OUTPUT_PATH / 'cnn1d_best.pth')
torch.save(model_b.state_dict(), OUTPUT_PATH / 'cnn_bilstm_best.pth')
torch.save(model_c.state_dict(), OUTPUT_PATH / 'cnn_attention_best.pth')

# Save results
results_to_save = {
    'baseline_f1': BASELINE_MACRO_F1,
    'cnn1d': {k: float(v) if isinstance(v, (float, np.floating)) else v for k, v in results_a.items()},
    'cnn_bilstm': {k: float(v) if isinstance(v, (float, np.floating)) else v for k, v in results_b.items()},
    'cnn_attention': {k: float(v) if isinstance(v, (float, np.floating)) else v for k, v in results_c.items()},
}

with open(OUTPUT_PATH / 'dl_results.json', 'w') as f:
    json.dump(results_to_save, f, indent=2)

print('✅ Models and results saved to outputs_dl/')
print(f'   - cnn1d_best.pth')
print(f'   - cnn_bilstm_best.pth')
print(f'   - cnn_attention_best.pth')
print(f'   - dl_results.json')